In [22]:
import numpy as np
import matplotlib as plt
from tabulate import tabulate

### Resolver o sistema:
 2x + y + 3z = 4
 -1x + 3y + 2z = 3
 3x + y - 3z = 2

In [25]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
ba = np.array([4, 3, 2])
Ai = np.linalg.inv(A)
r = Ai @ ba
r

array([0.81632653, 0.95918367, 0.46938776])

In [4]:
def cofat(m):
    n = len(m)
    cof = np.zeros((n, n))
    for col in range(n):
        for lin in range(n):
            d_ij = np.delete(m, lin, 0)
            d_ij = np.delete(d_ij, col, 1)
            det = np.linalg.det(d_ij)
            cof[lin][col] = (-1) ** (lin + col) * det
    return cof


def adjuta(m):
    return cofat(m).transpose()


def inv_cofat(m):
    return (1 / np.linalg.det(m)) * adjuta(m)

In [14]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
Ai1 = np.linalg.inv(A)
Ai2 = inv_cofat(A)
print("Ai com linalg:\n", Ai1, "\nAi com inv_cofat:\n", Ai2)

Ai com linalg:
 [[ 0.2244898  -0.12244898  0.14285714]
 [-0.06122449  0.30612245  0.14285714]
 [ 0.20408163 -0.02040816 -0.14285714]] 
Ai com inv_cofat:
 [[ 0.2244898  -0.12244898  0.14285714]
 [-0.06122449  0.30612245  0.14285714]
 [ 0.20408163 -0.02040816 -0.14285714]]


In [24]:
A = np.array([[2, 1, 3], [-1, 3, 2], [3, 1, -3]])
ba = np.array(([4, 3, 2]))
x1 = np.linalg.solve(A, ba)
x2 = inv_cofat(A) @ ba
print(x1, " = ", x2)

[0.81632653 0.95918367 0.46938776]  =  [0.81632653 0.95918367 0.46938776]


In [16]:
def det_cofat(M):
    n = len(M)
    if n == 1:
        return M
    cof = np.zeros(n)
    for col in range(n):
        d = np.delete(M, 0, 0)
        d = np.delete(d, col, 1)
        cof[col] = ((-1) ** col) * det_cofat(d)
    return M[0, :] @ cof

In [17]:
np.linalg.det(A), det_cofat(A)

(-49.000000000000014, -49.0)

In [18]:
def cramer(M, B, prt):
    n = len(M)
    c = np.c_[M, B]
    d = det_cofat(M)
    x = np.zeros(n)
    if prt:
        print("Matriz aumentada [C=A|b] det(A) = %f" % d)
        print(c)
    for k in range(0, n):
        ak = M.copy()
        ak[:, k] = B
        dk = det_cofat(ak)
        x[k] = dk/d
        if prt:
            print("Matriz A%d    det(A%1d) = %f" % (k, k, dk))
            print(ak)
            print("x(%d) = %f / %f = %f\n" % (k, dk, d, x[k]))
    return x

In [26]:
inv_cofat(A)@ba

array([0.81632653, 0.95918367, 0.46938776])

In [27]:
cramer(A, ba, True)

Matriz aumentada [C=A|b] det(A) = -49.000000
[[ 2  1  3  4]
 [-1  3  2  3]
 [ 3  1 -3  2]]
Matriz A0    det(A0) = -40.000000
[[ 4  1  3]
 [ 3  3  2]
 [ 2  1 -3]]
x(0) = -40.000000 / -49.000000 = 0.816327

Matriz A1    det(A1) = -47.000000
[[ 2  4  3]
 [-1  3  2]
 [ 3  2 -3]]
x(1) = -47.000000 / -49.000000 = 0.959184

Matriz A2    det(A2) = -23.000000
[[ 2  1  4]
 [-1  3  3]
 [ 3  1  2]]
x(2) = -23.000000 / -49.000000 = 0.469388



array([0.81632653, 0.95918367, 0.46938776])

In [28]:
def eliminar_linha(lin, p, c, prt):
  c = c.astype(float)
  [N, M] = c.shape
  if prt:
    print("(L%d)=(L%d)-(%f)/(%f)*L(%d)" % (lin, lin, c[lin, p], c[p, p], p))
  m = c[lin, p] / c[p, p]
  c[lin, p:M+1]=c[lin,p:M+1]-m*c[p,p:M+1]
  return c, m

def pivotar_coluna(p,l_ini,l_fim,c,prt):
     [N,M]=c.shape
     linhas = np.arange(l_ini,l_fim)
     dist=np.argmax(abs(c[linhas,p]))
     if dist!=0:  #troca linhas
         c[[ p, (dist+p)],:] = c[[ (dist+p) , p ],:]
         if prt:
             print("Trocando linhas %d e %d" % (p,dist+p))
             pdtabulate=lambda c:tabulate(c,headers='keys')
             print(pdtabulate(c))
     return c, dist

def substituicao_regressiva(a,b,prt):
  [N,M]=a.shape
  x = np.zeros(N)*np.nan
  if a[N - 1, N - 1]!=0: x[N - 1]= b[N - 1] / a[N - 1, N - 1];
  if prt:
      print("Substituição regressiva\nx(%d)=%f" % (N-1,x[N-1]))
  for lin in range(N-2,-1,-1):
     if a[lin, lin]!=0:
       x[lin]=(b[lin]-a[lin,lin+1:N] @ x[lin+1:N])/a[lin,lin]
     if prt:
         print("x(%d)=%f" % (lin,x[lin]))
  return x

In [29]:
def eliminacao_gauss(a, b, prt, pivot): # com pivotamento
   a = a.astype(float)
   b = b.astype(float)
   C=np.c_[a, b]
   [N, M]=C.shape
   x = np.zeros(N)
   if prt:
       print("Matriz Aumentada [C=A|b]")
       print(C)
   for p in range(N-1):
       if pivot:
         (C,dist)=pivotar_coluna(p,p,N,C,prt)
       elif C[p, p]== 0:
          break
       if prt:
            print("Eliminando coluna %d com Pivô %f" % (p,C[p,p]) )
       for lin in range(p+1,N): #eliminação progressiva
           (C,m)=eliminar_linha(lin,p,C,prt)
       if prt:
           print(C)
   if C[p, p]!=0:
         x=substituicao_regressiva(C[:,0:N],C[:,N],prt)
   else:
         print("Não há solução única pois matriz A é singular")
         x[1:N]= np.inf
   return x

In [33]:
A, ba

(array([[ 2,  1,  3],
        [-1,  3,  2],
        [ 3,  1, -3]]),
 array([4, 3, 2]))

In [34]:
eliminacao_gauss(A, ba, True, False)

Matriz Aumentada [C=A|b]
[[ 2.  1.  3.  4.]
 [-1.  3.  2.  3.]
 [ 3.  1. -3.  2.]]
Eliminando coluna 0 com Pivô 2.000000
(L1)=(L1)-(-1.000000)/(2.000000)*L(0)
(L2)=(L2)-(3.000000)/(2.000000)*L(0)
[[ 2.   1.   3.   4. ]
 [ 0.   3.5  3.5  5. ]
 [ 0.  -0.5 -7.5 -4. ]]
Eliminando coluna 1 com Pivô 3.500000
(L2)=(L2)-(-0.500000)/(3.500000)*L(1)
[[ 2.          1.          3.          4.        ]
 [ 0.          3.5         3.5         5.        ]
 [ 0.          0.         -7.         -3.28571429]]
Substituição regressiva
x(2)=0.469388
x(1)=0.959184
x(0)=0.816327


array([0.81632653, 0.95918367, 0.46938776])